# Running SGP4 to calculate Fermi geodetic coordinates for a specified time

### <span style="color:red; font-family:Georgia;">Robert Cameron, February 2016</span>

Based on the python implementation of SGP4 orbit propagator
available at https://pypi.python.org/pypi/sgp4/

In [169]:
import sys
import numpy as np
from subprocess import Popen, PIPE, STDOUT
from math import radians, sin, cos, sqrt, acos, asin, atan2, pi, degrees
import operator
sys.path.append('/Users/rc/Documents/GLAST/sgp4-1.4')
from sgp4.earth_gravity import wgs84
from sgp4.io import twoline2rv
from sgp4.ext import jday, invjday, days2mdhms

In [170]:
# specify the UTC date and time at which to calculate the Fermi position
date = "2015-11-15" 
time = "07:31:09.833"
time = "11:01:36.849"
#time = "13:01:27.288"
#time = "14:36:30.387"
#time = "16:21:22.232"

In [171]:
tle = ['1 33053U 08029A   15319.43445656  .00001507  00000-0  72237-4 0  9997',
'2 33053  25.5836 347.6463 0012945  60.0439 300.1380 15.10243055409555']

In [172]:
satellite = twoline2rv(tle[0], tle[1], wgs84)

In [173]:
(y,mon,d) = map(int, date.split("-"))
(h,m,s) = map(float, time.split(":"))
# calculate the Julian day from the UTC date and time
jd = jday(y,mon,d,h,m,s)
print (jd)

2457341.959454271


In [174]:
# compute Greenwich Sidereal Time (GST) in seconds
# then compute Local Sidereal Time (LST) at Longitude = 0
# for the constants used below, see Astronomical Algorithms by Jean Meeus, p. 84 (Eq. 11-4).
c = [280.46061837, 360.98564736629, 0.000387933, 38710000.0]
jd2000 = 2451545.0
t0 = jd - jd2000
t = t0/36525
theta = c[0] + (c[1] * t0) + t**2*(c[2] - t/ c[3])  # GST in seconds
lst = theta/15                                  # LST at longitude = 0, in hours 
lst = (lst + 24) % 24
print (lst)

14.641465619846713


In [175]:
(y,mon,d) = map(int, date.split("-"))
(h,m,s) = map(float, time.split(":"))
pos, vel = satellite.propagate(y,mon,d,h,m,s)

In [176]:
pos

(-3966.979641728793, 5269.563783180933, 2049.617160896793)

In [177]:
az_angle = atan2(pos[1],pos[0])       # azimuth angle (radians)
angle_sid = lst * 2.*pi/24.        # sidereal angle (radians)
lon = (az_angle - angle_sid ) % (2*pi)     # geo longitude (radians)
r = sqrt(pos[0]**2 + pos[1]**2)
lat = atan2(pos[2],r)       # geo latitude (radians)
glon = degrees(lon)
if glon > 180: 
    glon -= 360
glat = degrees(lat)
#print (date, time, ":", glon, glat)
print (date, time, ": %.3f %.3f" % (glon, glat))

2015-11-15 11:01:36.849 : -92.649 17.262


In [178]:
%%inactive
r = np.linalg.norm(pos)
glon = degrees(atan2(pos[1],pos[0]))
glat = degrees(asin(pos[2]/r))
print (glon, glat)

ERROR: Cell magic `%%inactive` not found.


In [179]:
%%inactive
ws = tle[0].split()
ydoy = float(ws[3])
year = int(ydoy)/1000 + 2000
jd0 = jday(year,1,1,0,0,0)
ylen = 365
if year % 4 == 0:
    ylen = 366
fypl = []
bpl = []
for doy in range(ylen):
    for hour in range(24):
# add a small amount to the day number to ensure it is always a fraction of a second past the hour
        fdoy = doy + hour/24.0 + 1.0e-9
        jd = jd0 + fdoy
        (y,mon,d,h,m,s) = invjday(jd)
        s = int(s)
        fy = y + fdoy/ylen
        fypl.append(fy)
        pos, vel = satellite.propagate(y,mon,d,h,m,s)
        pole = crossprod(pos,vel)
        polelen = sqrt(pole[0]*pole[0]+pole[1]*pole[1]+pole[2]*pole[2])
# calculate sun coordinates
        n = jd - 2451545.0
        L = (280.460 + 0.9856474 * n) % 360
        g = radians((357.528 + 0.9856003 * n) % 360)
        lamda = radians(L + 1.915 * sin(g) + 0.020 * sin(2*g))
        epsilon = radians(23.439 - 0.0000004 * n)
        Xsun = cos(lamda)
        Ysun = cos(epsilon) * sin(lamda)
        Zsun = sin(epsilon) * sin(lamda)
        dotprod = reduce( operator.add, map( operator.mul, (Xsun,Ysun,Zsun), pole))
        beta = degrees(asin(dotprod/polelen))
        bpl.append(beta)
#        print oformat % (y, mon, d, h, m, s, fy, pos[0], pos[1], pos[2], beta)

ERROR: Cell magic `%%inactive` not found.


In [180]:
vel

(-6.1900850794762485, -3.716942306962325, -2.3814911949331234)